In [1]:
import spcm
from spcm import units
import numpy as np

# card : spcm.Card

ip = 'TCPIP::192.168.1.83::INSTR'

card = spcm.Card(ip)

card.open(ip)

# setup card for DDS
card.card_mode(spcm.SPC_REP_STD_DDS)

# Setup the channels
channels = spcm.Channels(card)
channels.enable(True)
channels.output_load(50 * units.ohm)
channels.amp(1. * units.V)
card.write_setup()

# trigger mode
trigger = spcm.Trigger(card)
trigger.or_mask(spcm.SPC_TMASK_EXT0) # disable default software trigger
trigger.ext0_mode(spcm.SPC_TM_POS) # positive edge
trigger.ext0_level0(1.5 * units.V) # Trigger level is 1.5 V (1500 mV)
trigger.ext0_coupling(spcm.COUPLING_DC) # set DC coupling
card.write_setup()

# Setup DDS functionality
dds = spcm.DDS(card, channels=channels)
dds.reset()

# dds.data_transfer_mode(spcm.SPCM_DDS_DTM_DMA)

dds.trg_src(spcm.SPCM_DDS_TRG_SRC_CARD)
# dds.phase_behaviour(0)

core_list = [hex(2**n) for n in range(20)]

card.start(spcm.M2CMD_CARD_ENABLETRIGGER)

In [2]:
def split_array(f_split = 2.4e6, f_width_single_side = 4.e6, n_per_side = 4):
    f_center = 74.8e6
    # array1 = np.linspace( f_center-f_split/2-f_width_single_side ,f_center-f_split/2,n_per_side)
    array1 = f_center - f_split/2 - np.linspace( f_width_single_side , 0. ,n_per_side)
    # array2 = np.linspace( f_center+f_split/2+f_width_single_side ,f_center+f_split/2,n_per_side)
    array2 = f_center + f_split/2 + np.linspace( 0. , f_width_single_side ,n_per_side)
    # array1 = np.linspace(70.5e6,72.5e6,n)
    # array2 = np.linspace(77.5e6,79.5e6,n)
    return np.concatenate([array1,array2])

def compute_tweezer_1064_phases(n_tweezers,amplitudes):
    phase_tweezer_array = np.empty([n_tweezers])
    for tweezer_idx in range(n_tweezers):
        if tweezer_idx == 0:
            phase_tweezer_array[0] = 360
        else:
            phase_ij = 0
            for j in range(1,tweezer_idx):
                phase_ij = phase_ij + 2*np.pi*(tweezer_idx - j)*amplitudes[tweezer_idx]
            phase_i = (phase_ij % 2*np.pi) * 360
            phase_tweezer_array[tweezer_idx] = phase_i
    return phase_tweezer_array

# f_list = split_array(f_split=2.4e6)
f_list = [71.75e6,76.e6,80.e6]
# f_list = [73.6e6,76.e6]

n_tweezers = len(f_list)

# a_list = np.linspace(1/n_tweezers,1/n_tweezers,n_tweezers)
# a_list = [.1,.1,.1,.05,.1,.35]
a_list = [.48,.33,.18]

phases = compute_tweezer_1064_phases(len(f_list),a_list)

def normalize_alist(alist):
    sum = np.sum(alist)
    return a_list/sum
a_list = normalize_alist(a_list)

for tweezer_idx in range(len(core_list)):
    if tweezer_idx < len(f_list):
        dds[tweezer_idx].amp(a_list[tweezer_idx])
        dds[tweezer_idx].freq(f_list[tweezer_idx])
        dds[tweezer_idx].phase(phases[tweezer_idx])
    else:
        dds[tweezer_idx].amp(0.)
dds.exec_at_trg()
dds.write_to_card()
trigger.force()

In [3]:
# compute tweezer movement params
dpf = 5.135e-12 # meter per Hz

# which tweezer (give idx of desired tone in f_list)
which_tweezer = 0

# how far to move?
distance = 17.e-6 # m

# how many steps?
n_steps = 1000

# set acceleration of movement
acc = .00075 # meter / s^2
acc_f = acc / dpf

# time necessary to move half the desired distance (accelerate to here)
t_tot = np.sqrt(distance / (2*acc))

# how much time per linear ramp
dt = t_tot / n_steps

# change in slope of linear ramp per dt
df = acc_f*dt

# compute list of linear ramp slopes
acceleration_array = []
decceleration_array = []

for n in range(1,n_steps+1):
    acceleration_array.append(df*n)
    decceleration_array.append(df*(n_steps-n))

slopes_left = np.concatenate([np.array(acceleration_array),np.array(decceleration_array)])
slopes_right = -1.*slopes_left

In [4]:
# execute single movement

# start trigger timer, which outputs trigger events at a given rate
dds.trg_src(spcm.SPCM_DDS_TRG_SRC_TIMER)
dds.trg_timer(dt)
dds.exec_at_trg()
dds.write_to_card()

# write slopes to card
for slope in slopes_left:
    dds.frequency_slope(which_tweezer,slope)
    dds.exec_at_trg()
dds.write_to_card()

# reset trigger mode to external at the end
dds.trg_src(spcm.SPCM_DDS_TRG_SRC_CARD)
dds.exec_at_trg()
dds.write_to_card()


In [5]:
# move back to starting position

# start trigger timer, which outputs trigger events at a given rate
dds.trg_src(spcm.SPCM_DDS_TRG_SRC_TIMER)
dds.trg_timer(dt)
dds.exec_at_trg()
dds.write_to_card()

# write slopes to card
for slope in slopes_right:
    dds.frequency_slope(which_tweezer,slope)
    dds.exec_at_trg()
dds.write_to_card()

# reset trigger mode to external at the end
dds.trg_src(spcm.SPCM_DDS_TRG_SRC_CARD)
dds.exec_at_trg()
dds.write_to_card()

In [13]:
# oscillate between

oscillate = np.concatenate([slopes_right,slopes_left,slopes_left,slopes_right])

# start trigger timer, which outputs trigger events at a given rate
dds.trg_src(spcm.SPCM_DDS_TRG_SRC_TIMER)
dds.trg_timer(dt)
dds.exec_at_trg()
dds.write_to_card()

# write slopes to card
for slope in oscillate:
    dds.frequency_slope(which_tweezer,slope)
    dds.exec_at_trg()
dds.write_to_card()

# reset trigger mode to external at the end of the iteration
dds.trg_src(spcm.SPCM_DDS_TRG_SRC_CARD)
dds.exec_at_trg()
dds.write_to_card()

In [6]:
# card.close(card._handle)
card.stop()

card.close(card._handle)